# Tugas Event #2 - Solving Real-World Problems with Data Science

- Nama           : Hieronimus Jevon Valerian
- NIM            : 10819017
- Universitas    : Institut Teknologi Bandung
- Tanggal Mulai  : 07 November 2020
- Sumber Dataset : https://www.kaggle.com/c/titanic/data

## Titanic train.csv
- Sebenarnya data yang disediakan ada dua, yaitu train.csv dan test.csv
- Namun, untuk mempermudah analisis dan demi pemenuhan spesifikasi tugas (satu file dataset), maka akan digunakan data dari train.csv saja

## 1. Data Preparation & Observation

- Pada analisis kali ini, saya menggunakan dataset titanic. Data ini memberikan informasi tentang beberapa hal terkait  masing-masing penumpang beserta status "survived" (1) dan "died" (0).
- Nantinya dataset akan dibagi dengan proporsi 80% untuk training dan 20% untuk testing.
- Model akan dikembangkan menggunakan data train untuk belajar pola-pola yang muncul dalam memprediksi peluang seseorang akan "survived" atau "died"
- Setelah itu, model akan di deploy menggunakan data test untuk melihat seberapa akurat prediksi yang ditampilkan oleh model

In [1]:
#Import library
import pandas as pd

#Load dataset train.csv
df = pd.read_csv("train.csv")

In [2]:
#Melihat bentuk dataset
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Melihat info pada masing-masing kolom
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
#Statisitka Deskriptif dataset train.csv
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
#Melihat jumlah penumpang pria dan wanita
df["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [6]:
#Filtering rata-rata usia pada kolom "Survived" dan dikelompokkan berdasarkan jenis kelamin
for i in range(2):
    if i == 0:
        a = "meninggal"
    else:
        a = "selamat"
    
    b = round(df[(df["Sex"] == "male") & (df["Survived"] == i)]["Age"].mean(), 2)
    c = round(df[(df["Sex"] == "female") & (df["Survived"] == i)]["Age"].mean(), 2)
    
    print("Rata-rata usia pria {} = {} tahun".format(a, b))
    print("Rata-rata usia wanita {} = {} tahun".format(a, c))

Rata-rata usia pria meninggal = 31.62 tahun
Rata-rata usia wanita meninggal = 25.05 tahun
Rata-rata usia pria selamat = 27.28 tahun
Rata-rata usia wanita selamat = 28.85 tahun


1. Dataset terdiri dari 12 kolom dengan rincian 5 kolom bertipe int, 2 kolom bertipe float, dan 5 kolom bertipe object/string.
2. Terdapat null values pada kolom "Age", "Cabin", dan "Embarked" yang akan mengganggu proses perhitungan ke depannya.
3. Jumlah penumpang pria lebih banyak dari pada perempuan.
4. Usia pria yang meninggal berpusat di nilai yang lebih tinggi daripada yang selamat dan berlaku sebaliknya pada perempuan.
5. Ada kemungkinan usia penumpang laki-laki memang berdistribusi di nilai yang lebih tinggi daripada perempuan yang kemungkinan berdistribusi di nilai yang lebih rendah.
6. Data tidak berdistribusi pada rentang nilai yang sama dan akan berpengaruh pada penurunan akurasi model Machine Learning nantinya.
7. Sehingga perlu dilakukan beberapa proses pada dataset.

## 2. Data Visualization

In [7]:
#Import library
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

ModuleNotFoundError: No module named 'altair'

In [ ]:
#Visualisasi frekuensi penumpang berdasarkan "Survived" dan "Sex"
fig, ax = plt.subplots(figsize=(14, 14), nrows=2, ncols=2, sharey=True)
gender = ["male", "female"]

for i in range(2):
    for j in range(2):
        ax[i][j].hist(df[(df["Sex"] == gender[i]) & (df["Survived"] == j)]["Age"], bins=30)
        
        if j == 0:
            a = "Died"
        else:
            a = "Survived"
            
        ax[i][j].set_title("{} {} Distribution by Age".format(gender[i].title(), a))
        ax[i][j].set_xlabel("Age")
        ax[i][j].set_ylabel("Frequency")

plt.show()

- Terlihat bahwa distribusi usia untuk male dan female berdasarkan status "Survived" tidak berdistribusi normal
- Jumlah pria yang meninggal lebih banyak dibandingkan yang selamat dan mayoritas pria meninggal berusia di rentang 10 - 50 tahun-an
- Sedangkan pada wanita berlaku kebalikannya dengan mayoritas wanita selamat berusia rentang 10 - 45 tahun-an.

In [ ]:
#Melihat hubungan antara usia, fare, dan survived
fig, ax = plt.subplots(figsize=(30, 15))
mapbar = ax.scatter(df["Age"], df["Fare"],c=df["Survived"], s=200*df["Pclass"], alpha=0.8)

ax.set_title("Hubungan Usia Penumpang Dengan Ongkos yang Dikeluarkan")
ax.set_xlabel("Usia")
ax.set_ylabel("Fare ($)")
fig.colorbar(mapbar)

plt.show()

- Dari grafik dapat disimpulkan bahwa mayoritas penumpang yang meninggal adalah mereka yang membayar biaya perjalanan rendah.
- Dalam sistem kapal titanic, semakin rendah biaya perjalanan, semakin rendah lantai kamar yang mereka dapatkan.
- Ada kemungkinan alasan mereka meninggal adalah keterbatasan waktu untuk menyelamatkan diri sebelum air menyentuh lantai tempat mereka tinggal.

In [ ]:
# Distribusi usia penumpang berdasarkan pclass
alt.Chart(df,
         width=600,
         height=300,
         title="Distribusi Usia Penumpang"
         ).mark_bar(opacity=0.8).encode(x = alt.X("Age",
                                         title="Usia",
                                         axis=alt.AxisConfig(labelAngle=45)),
                               y = alt.Y("count()",
                                        title="Frekuensi Berdasarkan Usia"),
                               color = alt.Color("Pclass")
                               ).facet(column = alt.Column("Survived"))

- Ternyata mayoritas penumpang pada perjalanan tersebut berusia 15 - 45 tahun.
- Terlihat pula bahwa mayoritas penumpang yang meninggal (Survived = 0) mengambil Pclass = 3 (3rd class) yang merupakan class terendah.
- Hal ini sebanding dengan pernyataan kita di bagian sebelumnya bahwa penumpang yang meninggal mayoritas adalah mereka dengan biaya perjalanan rendah.
- Sehingga kemungkinan lantai tempat mereka tinggal berpotensi tenggelam lebih cepat dibandingkan 2nd class dan 1st class yang berada di tingkatan yang lebih tinggi.

## 3. Data Preprocessing

### 3.1 Handle Missing Value

In [8]:
#Melihat info pada masing-masing kolom
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [9]:
#Karena kolom cabin terlalu banyak memuat null value, maka akan kita drop
df = df.drop("Cabin", axis=1)

In [10]:
#Fill nilai nan pada column "Age" dengan nilai rata-rata column tersebut
df["Age"] = df["Age"].fillna(df["Age"].mean())

#Fill nilai nan pada column "Embarked" dengan metode ffill
df["Embarked"] = df["Embarked"].fillna(method="ffill")

In [11]:
#Mengecek keberadaan null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


### 3.2 Encoding Categorical Data

In [12]:
#Mendummies kolom "Sex" dan "Embarked"
df = pd.get_dummies(df, columns=["Sex", "Embarked"])

df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,0,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,1,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,0,1,0,0,1


### 3.3 Feature Centering and Scaling

In [13]:
#Karena data tidak berdistribusi pada rentang nilai yang sama, akan kita lakukan Normalisasi
#Import library
from sklearn.preprocessing import Normalizer

#Membuat objek model
norm_scaler = Normalizer()

#Memfitting model
col = ["SibSp", "Parch", "Pclass", "Age", "Fare"]
df[col] = norm_scaler.fit_transform(df[col])

In [14]:
df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,0.128322,"Braund, Mr. Owen Harris",0.941028,0.042774,0.0,A/5 21171,0.310112,0,1,0,0,1
1,2,1,0.012377,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.470345,0.012377,0.0,PC 17599,0.882309,1,0,1,0,0
2,3,1,0.109705,"Heikkinen, Miss. Laina",0.950778,0.000000,0.0,STON/O2. 3101282,0.289804,1,0,0,0,1
3,4,1,0.015720,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.550202,0.015720,0.0,113803,0.834735,1,0,0,0,1
4,5,0,0.083243,"Allen, Mr. William Henry",0.971173,0.000000,0.0,373450,0.223370,0,1,0,0,1


## 4. Model Development

### 4.1 Create Model

In [15]:
#Import library
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

#Memisahkan antara data feature dan data target (label)
x = df.drop(["Name", "Survived", "Ticket", "PassengerId"], axis=1)
y = df["Survived"]

#Mensplit antara data training dan data test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                   random_state = 21)

#Mengecek kombinasi terbaik untuk model
model = KNeighborsClassifier()
param_grid = {"n_neighbors" : np.arange(5, 101),
             "weights" : ["uniform", "distance"]}

gscv = GridSearchCV(model, param_grid=param_grid, scoring="roc_auc", cv=10)
gscv.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100]),
                         'weights': ['uniform', 'distance']},
             scoring='roc_auc')

In [16]:
#Melihat hasil kombinasi terbaik yang disarankan
gscv.best_params_

{'n_neighbors': 20, 'weights': 'uniform'}

In [17]:
#Melihat skor tertinggi dari kombinasi yang disarankan
gscv.best_score_

0.8448047425825204

In [18]:
#Membuat objek model
model = KNeighborsClassifier(n_neighbors=20, weights="uniform")

#Fitting model
m = model.fit(x_train, y_train)

### 4.2 Model Test

In [19]:
#Melakukan prediksi dengan model yang telah kita develop
y_predict = model.predict(x_test)

In [20]:
#Membuat dataframe baru dari hasil prediksi
result = pd.DataFrame(x_test.index)
result["Survived"] = y_predict
result.columns = ["PassengerId", "Survived"]

result.head()

,PassengerId,Survived
0,617,1
1,379,0
2,725,0
3,826,0
4,450,0


In [21]:
#Melihat jumlah penumpang selama dan meninggal
result["Survived"].value_counts()

0    112
1     67
Name: Survived, dtype: int64

- Model memprediksi jika ada perjalanan kapal titanic dengan rincian informasi seperti pada dataset x_test, akan ada 112 orang yang meninggal dan 67 orang yang selamat

In [22]:
#Menglihat seberapa akurat prediksi model kita menggunakan auc scoring
accuracy_score(y_test, y_predict)

0.770949720670391

In [23]:
#Menyimpang model dalam format .pickle
pickle.dump(m, open('m_model.pickle', 'wb'))

#Menyimpan data hasil prediksi dalam format csv
df.to_csv("result.csv", index=False)

## 5. Presentation

- Dengan menggunakan library KNeighborsClassifier, model kami diset untuk bisa memprediksi status seorang penumpang kapal titanic berdasarkan beberapa informasi yang tersedia pada dataset.
- Setelah melalui beberapa tahap pengembangan dan Manual Hyperparameter Tuning, model kami memiliki akurasi prediksi di angka 0.7709 atau sekitar 77.09%
- Menurut kami, angka ini sudah cukup baik dan mungkin dengan beberapa penyesuaian pada dataset dan Hyperparameter Tuning yang lebih detail bisa meningkatkan akurasi dari model kami.